In [1]:
import helper

Using TensorFlow backend.


In [2]:
BASE_PATH = '/home/dev/Documents/disser/'
DATA_PATH = BASE_PATH + 'compcars/data/'
IMAGE_DATA = 'image/'
OUTPUT_DATA_PATH = BASE_PATH + 'keras_compcars_all_distinct_classes/'

In [3]:
num_classes = 4455
num_epochs = 2000
batch_size = 8

In [4]:
train, test = helper.get_generator(OUTPUT_DATA_PATH, batch_size = batch_size)

Found 127645 images belonging to 4455 classes.
Found 9081 images belonging to 4455 classes.


In [5]:
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [6]:
xcep = InceptionV3(include_top=False, weights='imagenet')

In [7]:
x = xcep.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [8]:
model = Model(inputs=xcep.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_acc', patience=50)
model_cp = ModelCheckpoint('cars_cat_hinge.hdf5', save_best_only=True)
reduce_lr = ReduceLROnPlateau()

In [ ]:
model.fit_generator(
    callbacks=[early_stop, model_cp, reduce_lr],
    generator=train, 
    epochs=num_epochs, 
    steps_per_epoch=int(train.samples / batch_size),
    validation_data=test,
    validation_steps=int(test.samples / batch_size))

Epoch 1/2000
15955/15955 [==============================] - 4586s - loss: 8.1070 - acc: 0.0012 - val_loss: 8.3843 - val_acc: 0.0020
Epoch 2/2000
15955/15955 [==============================] - 4583s - loss: 7.8679 - acc: 0.0020 - val_loss: 8.0392 - val_acc: 0.0020
Epoch 3/2000
15955/15955 [==============================] - 4618s - loss: 7.7005 - acc: 0.0024 - val_loss: 8.1143 - val_acc: 0.0025
Epoch 4/2000
15955/15955 [==============================] - 4605s - loss: 7.4761 - acc: 0.0042 - val_loss: 7.6673 - val_acc: 0.0044
Epoch 5/2000
15955/15955 [==============================] - 4600s - loss: 7.1976 - acc: 0.0068 - val_loss: 7.5067 - val_acc: 0.0077
Epoch 6/2000
15955/15955 [==============================] - 4602s - loss: 6.8970 - acc: 0.0101 - val_loss: 7.2286 - val_acc: 0.0096
Epoch 7/2000
15955/15955 [==============================] - 4596s - loss: 6.6370 - acc: 0.0145 - val_loss: 6.9537 - val_acc: 0.0131
Epoch 8/2000
15955/15955 [==============================] - 4587s - loss: 6.

15955/15955 [==============================] - 4613s - loss: 1.7680 - acc: 0.5367 - val_loss: 6.3306 - val_acc: 0.1826
Epoch 63/2000
15955/15955 [==============================] - 4597s - loss: 1.7667 - acc: 0.5363 - val_loss: 6.3299 - val_acc: 0.1830
Epoch 64/2000
15955/15955 [==============================] - 4589s - loss: 1.7753 - acc: 0.5345 - val_loss: 6.2845 - val_acc: 0.1831
Epoch 65/2000
15955/15955 [==============================] - 4584s - loss: 1.7725 - acc: 0.5353 - val_loss: 6.2338 - val_acc: 0.1849
Epoch 66/2000
15955/15955 [==============================] - 4583s - loss: 1.7687 - acc: 0.5374 - val_loss: 6.3232 - val_acc: 0.1832
Epoch 67/2000
15955/15955 [==============================] - 4584s - loss: 1.7701 - acc: 0.5379 - val_loss: 6.2207 - val_acc: 0.1851
Epoch 68/2000
15955/15955 [==============================] - 4588s - loss: 1.7681 - acc: 0.5385 - val_loss: 6.2396 - val_acc: 0.1846
Epoch 69/2000
15955/15955 [==============================] - 4596s - loss: 1.7698 -